# 数值分析 实验5
2019011265 计93 丁韶峰

In [2]:
import numpy as np

幂法

In [3]:
def pow_method(A):
  n = A.shape[0]
  v = np.ones(n)
  old_lambda = 0
  while True:
    v = np.dot(A, v)
    new_lambda = v[np.argmax(np.abs(v))]
    v = v / new_lambda
    if (np.abs(new_lambda - old_lambda) < 1e-5):
      return new_lambda, v
    old_lambda = new_lambda

In [4]:
A = np.array([[5, -4, 1], [-4, 6, -4], [1, -4, 7]])
B = np.array([[25, -41, 10, -6], [-41, 68, -17, 10], [10, -17, 5, -3], [-6, 10, -3, 2]])
lambda_a, v_a = pow_method(A)
lambda_b, v_b = pow_method(B)
print("A lambda {} v {}".format(lambda_a, v_a))
print("B lambda {} v {}".format(lambda_b, v_b))

A lambda 12.254320584751564 v [-0.67401981  1.         -0.88955964]
B lambda 98.52169772379699 v [-0.60397234  1.         -0.25113513  0.14895345]


In [85]:
def householder(x):
  sign = 1 if x[0] >= 0 else -1
  sigma = sign * np.linalg.norm(x)
  
  if np.abs(sigma - x[0]) < 1e-8:
    return None
  y = np.copy(x)
  # print(x)
  # print(y)
  y[0] += sigma
  return y

In [86]:
def QR(A):
  n = A.shape[0]
  R = np.copy(A)
  Q = np.identity(n)
  for k in range(n - 1):
    R0 = R[k:, k:]
    v = householder(R0[:, 0])
    if v is None:
      continue
    w = v / np.linalg.norm(v)
    w = w.reshape((1, len(w)))
    H = np.identity(n)
    H[k:, k:] = np.identity(n - k) - (2 * np.matmul(w.transpose(), w))
    Q = np.matmul(Q, H)
    # print(Q)
    beta = np.dot(v.transpose(), v)
    for j in range(n - k):
      gamma = np.dot(v.transpose(), R0[:, j])
      # print(gamma)
      R0[:, j] -= 2 * gamma * v / beta
  return Q, R

In [87]:
def is_quasi_diag(A):
  n = A.shape[0]
  is_zero = A < 1e-6
  i = 0
  while i < n:
    is_zero[i, i] = True
    if i < n - 1 and is_zero[i + 1, i] == False:
      is_zero[i + 1, i] = True
      i += 2
    else:
      i += 1
  for i in range(n):
    for j in range(i):
      if not is_zero[i, j]:
        return False
  return True

In [88]:
def get_eigenvalue(A):
  n = A.shape[0]
  eigenvalue = np.zeros(n, dtype=np.complex128)
  i = 0
  while i < n:
    if i < n - 1 and A[i + 1, i] > 1e-6:
      eigenvalue[i : i + 2] = np.linalg.eig(A[i:i + 2, i:i + 2])[0]
      i += 2
    else:
      eigenvalue[i] = A[i, i]
      i += 1
  return eigenvalue

In [99]:
def QR_iter(A):
  n = A.shape[0]
  cnt = 0
  while True:
    Q, R = QR(A)
    # print(Q)
    # print(R)
    new_A = np.matmul(R, Q)
    cnt += 1
    # print(new_A)
    if is_quasi_diag(new_A):
      break
    if np.max((np.abs(new_A - A))) < 1e-6:
      print("QR coverges in {} steps, can't get eigenvalues".format(cnt))
      return new_A, None
    A = new_A
  print("QR coverges in {} steps".format(cnt))
  return A, get_eigenvalue(A)
    

In [105]:
def QR_shift_iter(A):
  n = A.shape[0]
  k = n
  cnt = 0
  while k > 1 and np.abs(A[k - 1, k - 2]) > 1e-6:
    old_A = np.copy(A)
    s = A[k - 1, k - 1]
    Q, R = QR(A[:k, :k] - s * np.identity(k))
    A[:k, :k] = np.matmul(R, Q) + s * np.identity(k)
    cnt += 1
    if is_quasi_diag(A):
      break
    if np.max(np.abs(A - old_A) < 1e-6):
      print("shifted QR coverges in {} steps, can't get eigenvalues".format(cnt))
  print("shifted QR coverges in {} steps".format(cnt))
  return A, get_eigenvalue(A)


In [106]:
A = np.matrix([[0.5, 0.5, 0.5, 0.5], [0.5, 0.5, -0.5 , -0.5], [0.5, -0.5, 0.5, -0.5], [0.5, -0.5, -0.5, 0.5]])
final_A, eig_A = QR_iter(A)
print(final_A)
print(eig_A)

[[-0.5 -0.5 -0.5 -0.5]
 [-0.5 -0.5  0.5  0.5]
 [-0.5  0.5 -0.5  0.5]
 [-0.5  0.5  0.5 -0.5]]
[[-1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -1.00000000e+00  1.11022302e-16  1.11022302e-16]
 [ 0.00000000e+00  0.00000000e+00 -1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.00000000e+00]]
[[ 0.5  0.5  0.5  0.5]
 [ 0.5  0.5 -0.5 -0.5]
 [ 0.5 -0.5  0.5 -0.5]
 [ 0.5 -0.5 -0.5  0.5]]
QR coverges in 1 steps, can't get eigenvalues
[[ 0.5  0.5  0.5  0.5]
 [ 0.5  0.5 -0.5 -0.5]
 [ 0.5 -0.5  0.5 -0.5]
 [ 0.5 -0.5 -0.5  0.5]]
None


In [104]:
shifted_final_A, shifted_eig_A = QR_shift_iter(A)
print(shifted_final_A)
print(shifted_eig_A)

QR coverges in 3 steps
[[-1.00000000e+00 -1.77392636e-06  1.02438497e-06 -7.24404755e-07]
 [-1.77392636e-06  1.00000000e+00  9.08451112e-13 -6.42446563e-13]
 [ 1.02438497e-06  9.08519774e-13  1.00000000e+00  3.71258381e-13]
 [-7.24404755e-07 -6.42476638e-13  3.71108492e-13  1.00000000e+00]]
[-1.+0.j  1.+0.j  1.+0.j  1.+0.j]
